In [1]:
import pandas as pd
import plotly.graph_objects as go
from utils import *

co_df = pd.read_csv("data/co_data.csv")
map_df = pd.read_csv("data/map_data.csv")
user_df = pd.read_csv("data/user_data.csv", index_col= 'userName')
replay_df = pd.read_csv("data/replay_data.csv", index_col= 'replayId')
pick_df = pd.read_csv("data/pick_data.csv", index_col= 'replayId')
ban_df = pd.read_csv("data/ban_data.csv", index_col= 'replayId')

In [2]:
# Filter high ranked players, >= 1400 represents 25%, top quartile
highranked_users = user_df[(user_df['stdScore'] >= 1400) | (user_df['fogScore'] >= 1400)]

# Replays played by high ranked players, exclude time out and draws
highranked_replays = replay_df.loc[pick_df[pick_df['userName'].isin(highranked_users.index)].index.unique()]
highranked_replays = highranked_replays[(highranked_replays['isBoot'] != True) & 
                                        (highranked_replays['winnerPlayerIndex'] != 0)]

In [3]:
CO_NAME = 'Gage'

available_matches = get_co_available_replays(CO_NAME, ban_df, highranked_replays.index)
banned_matches = get_co_banned_replays(CO_NAME, ban_df, highranked_replays.index)
picked_matches = get_co_picked_replays(CO_NAME, pick_df, highranked_replays.index)

pickrate = get_co_pick_rate(CO_NAME, pick_df, ban_df, highranked_replays.index)
banrate = get_co_ban_rate(CO_NAME, ban_df, highranked_replays.index)


In [4]:
co_matchup_total = get_co_matchup_total(CO_NAME, pick_df, highranked_replays.index)
co_matchup_loss = get_co_matchup_loss(CO_NAME, pick_df, highranked_replays.index, highranked_replays)
co_matchup_win = get_co_matchup_win(CO_NAME, pick_df, highranked_replays.index, highranked_replays)
co_matchup_win_rate = get_co_matchup_win_rate(CO_NAME, pick_df, highranked_replays.index, highranked_replays)


In [19]:
co_list = co_df['coName'].values
matchup_table_dict = {}

In [20]:
for co_name in co_list:
    matchup_table_dict[co_name] = get_co_matchup_table(co_name, pick_df, highranked_replays.index, highranked_replays)

In [136]:
co_pick_frequency = pick_df.loc[highranked_replays.index, :]['coName'].value_counts()
co_pick_frequency = co_pick_frequency.drop('(No CO)')

In [138]:
all_co_matchup_table = pd.concat(matchup_table_dict.values(), keys= matchup_table_dict.keys())

# Removing (No CO)
all_co_matchup_table = all_co_matchup_table.drop(index= '(No CO)', level= 0)
all_co_matchup_table = all_co_matchup_table.drop(index= '(No CO)', level= 1)

# Sorting CO alphabetically
# all_co_matchup_table = all_co_matchup_table.sort_index()

# Sorting CO by pick frequency
all_co_matchup_table = all_co_matchup_table.reindex(co_pick_frequency.index, level= 0)
all_co_matchup_table = all_co_matchup_table.reindex(co_pick_frequency.index, level= 1)

In [139]:
co_matchup_matrix = all_co_matchup_table.unstack(1)

In [170]:
fig = go.Figure(
    go.Heatmap(
    z= co_matchup_matrix['matchupWinRate'].to_numpy(),
    x= co_matchup_matrix['matchupWinRate'].columns,
    y= co_matchup_matrix['matchupWinRate'].index,
    customdata= co_matchup_matrix['matchupTotal'].to_numpy(),
    colorscale= 'emrld',
    texttemplate= "%{z:.2f}",
    hovertemplate= (
        "matchupWinRate: %{z:.2f}<br>"
        "matchupTotal: %{customdata}"
        "<extra></extra>")
    )
)

fig.update_xaxes(
    title= "Losing CO",
    side= 'top',
)
fig.update_yaxes(
    title= "Winning CO",
    autorange= 'reversed',
)
fig.update_layout(
    title= ("<b>CO Matchup Matrix</b></i>"),
    height= 1000,
    width= 1000,
    margin=dict(l=125, r=50, t=125, b=50),
)

fig.show()

In [172]:
fig.write_html("co_matchup_matrix.html")

In [6]:
# Why can't gage win vs penny?
# This is an abomination
pick_df.loc[pick_df.loc[picked_matches, :][pick_df.loc[picked_matches, :]['coName'] == 'Penny'].index, :]

,playerIndex,coName,userName
replayId,,,
6017,1,Penny,Plague
6017,2,Gage,Mini Nini
5719,1,Gage,PhantomFullForce
5719,2,Penny,Sliicer
5078,1,Gage,BWAVE
5078,2,Penny,BusterBeachside
4789,1,Gage,Chan ' tcho
4789,2,Penny,DC290
4558,1,Gage,Tee Lord


In [7]:
replay_df.loc[4169]

winnerPlayerIndex             1
turns                        18
actions                     835
mapId                       110
isStd                      True
isBoot                    False
mapName              Swamp City
winnerCoName               Gage
winnerUserName           Mauwus
Name: 4169, dtype: object